In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD, Nadam, RMSprop
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.regularizers import l1, l2

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.cnn3 import cnn, cnn_regularized

Using Theano backend.
Using gpu device 1: GeForce GTX 680 (CNMeM is disabled, cuDNN 5005)


In [2]:
lr=0.005
l1 = 0.00001
l2 = 0.00001
dropout = 0.5
n_epochs=100
batch_size=32
input_shape=(140, 140, 3)

name = 'cnn_140_rgb_corrected_lr_%f_sgd_he_normal__l1_%f_l2_%f_dropout_%f_r' % (lr, l1, l2, dropout)

In [3]:
print('loading model...')
# model = cnn(input_shape=input_shape, init='he_normal')
model = cnn_regularized(input_shape=input_shape, init='he_normal', l1=l1, l2=l2)
model.summary()

optimizer = SGD(lr=lr, clipnorm=4., nesterov=True)
# optimizer = Nadam(lr=lr)
# optimizer = RMSprop(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

csv_logger = CSVLogger('%s_training.log' % name, append=True)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 140, 140, 128) 18944       input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 140, 140, 128) 0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 70, 70, 128)   0           activation_1[0][0]               
__________________________________________________________________________

In [4]:
print('Initializing data generators...')
train_set_file = '/home/tanuj/Workspace/power-grid-detection/dataset/corrected/19/train_data.csv'
validation_set_file = '/home/tanuj/Workspace/power-grid-detection/dataset/corrected/19/validation_data.csv'
test_set_file = '/home/tanuj/Workspace/power-grid-detection/dataset/corrected/19/test_data.csv'

train_data_gen = DataGenerator(dataset_file=train_set_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=validation_set_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=test_set_file, batch_size=batch_size)
print('done.')

Initializing data generators...
done.


In [5]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer, current_model_checkpointer],
                              nb_worker=2)
print('done.')

Fitting model...
Epoch 1/100
10496/10496 [==============================] - 347s - loss: 2.8537 - acc: 0.6614 - val_loss: 0.6254 - val_acc: 0.6734
Epoch 2/100
10496/10496 [==============================] - 348s - loss: 2.7859 - acc: 0.7001 - val_loss: 0.3949 - val_acc: 0.8279
Epoch 3/100
10496/10496 [==============================] - 345s - loss: 2.5575 - acc: 0.8647 - val_loss: 0.1767 - val_acc: 0.9476
Epoch 4/100
10496/10496 [==============================] - 345s - loss: 2.3547 - acc: 0.9510 - val_loss: 0.1147 - val_acc: 0.9696
Epoch 5/100
10496/10496 [==============================] - 344s - loss: 2.3141 - acc: 0.9628 - val_loss: 0.1211 - val_acc: 0.9674
Epoch 6/100
10496/10496 [==============================] - 345s - loss: 2.2990 - acc: 0.9659 - val_loss: 0.1025 - val_acc: 0.9714
Epoch 7/100
10496/10496 [==============================] - 345s - loss: 2.2899 - acc: 0.9675 - val_loss: 0.0948 - val_acc: 0.9714
Epoch 8/100
10496/10496 [==============================] - 344s - loss: 2

In [ ]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])